# Offline monitoring

This Jupyter Notebook helps to visualize the L60 data. This version enables the visualization of time evolution of given monitoring parameters. 

In [ ]:
%matplotlib widget
import os, sys, json
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import dates
import pickle as pkl
import ipywidgets as widget
import ipydatetime
import datetime
import numpy as np
from datetime import datetime, timezone

def get_day_hour(date):
    if "T" in date:
        start = date.split('_')[0][:-1]
        stop = date.split('_')[1][:-1]
        start_day = datetime.strptime(start,"%Y%m%dT%H%M%S").strftime(f"%Y/%m/%d %H:%M")
        stop_day = datetime.strptime(stop,"%Y%m%dT%H%M%S").strftime(f"%Y/%m/%d %H:%M")
        time_interval = start_day+" -> "+stop_day
    else:
        time_interval = "no time cuts"
    
    return time_interval

## Load data

First, let's load data separately for HPGe detectors (geds), SiPMs (smps) and channel 0 (that collects HW pulser and FC trigger events too).

In [ ]:
exp         = 'l60'
period      = 'p01'
datatype    = 'phy'
foldername  = 'out'

pkl_name = exp + '-' + period

pkl_files = os.listdir(f'{foldername}/pkl-files/par-vs-time/')
pkl_files = [file for file in pkl_files if 'pkl' in file] 

runs = list(dict.fromkeys([file.split('-')[2] for file in pkl_files]))
print("Available runs: ", runs)

geds_list = [file for file in pkl_files if 'string' in file if 'pulser' not in file]      # geds: grouped by string
spms_list = [file for file in pkl_files if 'string' not in file if 'pulser' not in file]  # spms: grouped by barrel
ch000_list = [file for file in pkl_files if 'pulser' in file] 

# geds
if geds_list != []:
    # configuration
    geds_map = list(dict.fromkeys([file.split('-')[-1].split('.')[0] for file in geds_list]))
    # parameters
    geds_par = list(dict.fromkeys([file.split('-')[-2] for file in geds_list]))
    # date
    geds_date = sorted(list(dict.fromkeys([file.split('-')[4] for file in geds_list])))
    print("\ngeds strings:", geds_map)
    print("geds parameters:", geds_par)
else: 
    geds_map = geds_par = geds_date = []
    print("\n-> NO data for geds were found")

geds_date_formatted = [get_day_hour(date) for date in geds_date]
geds_time_option = [(key, value) for key,value in zip(geds_date_formatted, geds_date)]
if "no time cuts" in geds_date_formatted:
    geds_time_option = [('all','no_time_cuts')]
    geds_date = ['no_time_cuts']
print("geds dates:", geds_date)

# spms
if spms_list != []:
    # configuration
    spms_map  = list(dict.fromkeys([file.split('-')[-1].split('.')[0] for file in spms_list]))
    # parameters
    spms_par  = list(dict.fromkeys([file.split('-')[-2] for file in spms_list]))
    # dates
    spms_date = sorted(list(dict.fromkeys([file.split('-')[4] for file in spms_list])))
    print("\nspms barrels:", spms_map)
    print("spms parameters:", spms_par)
    print("qui:", spms_date)
else:    
    spms_map = spms_par = spms_date = []
    print("\n-> NO data for spms were found")   

spms_date_formatted = [get_day_hour(date) for date in spms_date]
spms_time_option = [(key, value) for key,value in zip(spms_date_formatted, spms_date)]
if "no time cuts" in spms_date_formatted:
    spms_time_option = [('all','no_time_cuts')]
    spms_date = ['no_time_cuts']
print("spms dates:", spms_date)
    

# ch000
if ch000_list != []:
    # parameters
    ch000_par = list(dict.fromkeys([file.split('-')[-2] for file in ch000_list]))
    # date
    ch000_date = sorted(list(dict.fromkeys([file.split('-')[4] for file in ch000_list])))
    print("\nch000 parameters:", ch000_par)
else: 
    ch000_par = ch000_date = []
    print("\n-> NO data for ch000 were found")

ch000_date_formatted = [get_day_hour(date) for date in ch000_date]
ch000_time_option = [(key, value) for key,value in zip(ch000_date_formatted, ch000_date)]
if "no time cuts" in ch000_date_formatted:
    ch000_time_option = [('all','no_time_cuts')]
    ch000_date = ['no_time_cuts']

## Interactive widgets
Define here the interactive widgets you want to include in the final plots. 

In [ ]:
if geds_date!=[]:
    # select widget for selecting the date (for geds) 
    geds_time_select = widget.Select(
        options=geds_time_option,
        value=geds_date[0],
        description='Time:',
        disabled=False,
        layout={'width': 'max-content'}
)
if spms_date!=[]:
    # select widget for selecting the date (for spms)
    spms_time_select = widget.Select(
        options=spms_time_option,
        value=spms_date[0],
        description='Time:',
        disabled=False,
        layout={'width': 'max-content'}
)
if ch000_date!=[]:
    # select widget for selecting the date (for ch000)
    ch000_time_select = widget.Select(
        options=ch000_time_option,
        value=ch000_date[0],
        description='Time:',
        disabled=False,
        layout={'width': 'max-content'}
)


# tab for selecting parameters to plot (for geds)
par_geds_buttons = widget.ToggleButtons(
    options= geds_par,
    description='Parameter:',
    disabled=False,
    button_style='',
    tooltips=[]
)
# tab for selecting parameters to plot (for spms)
par_spms_buttons = widget.ToggleButtons(
    options= spms_par,
    description='Parameter:',
    disabled=False,
    button_style='',
    tooltips=[]
)
# tab for selecting parameters to plot (for ch000)
par_ch000_buttons = widget.ToggleButtons(
    options= ch000_par,
    description='Parameter:',
    disabled=False,
    button_style='',
    tooltips=[]
)

# par-vs-time
geds_map_buttons = widget.ToggleButtons(
    options= geds_map,
    description='String #:',
    disabled=False,
    button_style='',
    tooltips=[],
) 
spms_map_buttons = widget.ToggleButtons(
    options= spms_map, 
    description='Position:',
    disabled=False,
    button_style='',
    tooltips=[],
) 

# run widget
run_button = widget.ToggleButtons(
    options= runs,
    description='Run:',
    disabled=False,
    button_style='',
    tooltips=[]
)

# Germanium detectors

Pick the time interval you want to inspect from the left panel; run, parameter and string number to inspect can be selected on the right side of time panel.

In [ ]:
def plot_geds(time_select, parameter, string, run):
    if time_select == 'no_time_cuts':
        ax = pkl.load(open(f'{foldername}/pkl-files/par-vs-time/{pkl_name}-{run}-{datatype}-{parameter}-{string}.pkl', 'rb'))
    else:
        ax = pkl.load(open(f'{foldername}/pkl-files/par-vs-time/{pkl_name}-{run}-{datatype}-{time_select}-{parameter}-{string}.pkl', 'rb'))
    plt.tight_layout()
    plt.show()
    
if geds_par == []:
    print(f'There are no parameters to plot!')
    sys.exit()
out_geds = widget.interactive_output(plot_geds, {'time_select':geds_time_select,
                                                 'parameter'  :par_geds_buttons, 
                                                 'string'     :geds_map_buttons,
                                                 'run': run_button})

widget.VBox([widget.HBox([geds_time_select,widget.VBox([run_button, par_geds_buttons, geds_map_buttons])]),out_geds])

# SiPM detectors 

Pick the time interval you want to inspect from the left panel; run, parameter and barrel to inspect can be selected on the right side of time panel.

In [ ]:
def plot_spms(time_select, parameter, string, run):
    if time_select == 'no_time_cuts':
        ax = pkl.load(open(f'{foldername}/pkl-files/par-vs-time/{pkl_name}-{run}-{datatype}-{parameter}-{string}.pkl', 'rb'))
    else:
        ax = pkl.load(open(f'{foldername}/pkl-files/par-vs-time/{pkl_name}-{run}-{datatype}-{time_select}-{parameter}-{string}.pkl', 'rb'))
    plt.tight_layout()
    plt.show()
    
if spms_par == []:
    print(f'There are no parameters to plot!')
    sys.exit()
out_spms = widget.interactive_output(plot_spms, {'time_select':spms_time_select,
                                                 'parameter'  :par_spms_buttons, 
                                                 'string'     :spms_map_buttons,
                                                 'run': run_button})

widget.VBox([widget.HBox([spms_time_select,widget.VBox([run_button, par_spms_buttons, spms_map_buttons])]),out_spms])

# ch000

Pick the time interval you want to inspect from the left panel; run and parameter to inspect can be selected through widgets on the right side of time panel.

In [ ]:
def plot_ch000(time_select, parameter, run):
    if time_select == 'no_time_cuts':
        ax = pkl.load(open(f'{foldername}/pkl-files/par-vs-time/{pkl_name}-{run}-{datatype}-{parameter}-pulser.pkl', 'rb'))
    else:
        ax = pkl.load(open(f'{foldername}/pkl-files/par-vs-time/{pkl_name}-{run}-{datatype}-{time_select}-{parameter}-pulser.pkl', 'rb'))
    ax.set_xlabel('day/month - time (UTC)')
    plt.setp(ax.get_xticklabels(), visible=True)
    plt.tight_layout()
    plt.show()
    
if ch000_par == []:
    print(f'There are no parameters to plot!')
    sys.exit()   
out_ch000 = widget.interactive_output(plot_ch000, {'time_select': ch000_time_select,
                                                 'parameter'  : par_ch000_buttons,
                                                 'run': run_button})

widget.VBox([widget.HBox([ch000_time_select,widget.VBox([run_button, par_ch000_buttons])]),out_ch000])